# Huggingface SageMaker-SDK - BERT Japanese NER example


1. [Introduction](#Introduction)  
2. [Development Environment and Permissions](#Development-Environment-and-Permissions)
    1. [Installation](#Installation)  
    2. [Permissions](#Permissions)
    3. [Uploading data to sagemaker_session_bucket](#Uploading-data-to-sagemaker_session_bucket)  
3. [Fine-tuning & starting Sagemaker Training Job](#Fine-tuning-\&-starting-Sagemaker-Training-Job)  
    1. [Creating an Estimator and start a training job](#Creating-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Download fine-tuned model from s3](#Download-fine-tuned-model-from-s3)
    4. [Named Entity Recognition on Local](#Named-Entity-Recognition-on-Local)  
4. [_Coming soon_:Push model to the Hugging Face hub](#Push-model-to-the-Hugging-Face-hub)

# Introduction

このnotebookは書籍：[BERTによる自然言語処理入門 Transformersを使った実践プログラミング](https://www.ohmsha.co.jp/book/9784274227264/)の[固有表現抽出(Named Entity Recognition)](https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb)をAmazon SageMakerで動作する様に変更を加えたものです。    
データは[ストックマーク株式会社](https://stockmark.co.jp/)さんで作成された[Wikipediaを用いた日本語の固有表現抽出データセット](https://github.com/stockmarkteam/ner-wikipedia-dataset)を使用します。    

このデモでは、AmazonSageMakerのHuggingFace Estimatorを使用してSageMakerのトレーニングジョブを実行します。    

_**NOTE: このデモは、SagemakerNotebookインスタンスで動作検証しています**_    

# Development Environment and Permissions


## Installation

このNotebookはSageMakerの`conda_pytorch_p36`カーネルを利用しています。    
日本語処理のため、`transformers`ではなく`transformers[ja]`をインスールします。

**_Note: このnotebook上で推論テストを行う場合、（バージョンが古い場合は）pytorchのバージョンアップが必要になります。_**

In [ ]:
# localで推論のテストを行う場合(CPU)
!pip install torch==1.7.1
# localで推論のテストを行う場合(GPU)
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install "sagemaker>=2.31.0" "transformers[ja]==4.6.1" "datasets[s3]==1.6.2" --upgrade

## Permissions

ローカル環境でSagemakerを使用する場合はSagemakerに必要な権限を持つIAMロールにアクセスする必要があります。[こちら](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください

In [ ]:
import sagemaker

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

# データの準備

[ストックマーク株式会社](https://stockmark.co.jp/)さんで作成された[Wikipediaを用いた日本語の固有表現抽出データセット](https://github.com/stockmarkteam/ner-wikipedia-dataset)をダウンロードします。

In [ ]:
!git clone --branch v2.0 https://github.com/stockmarkteam/ner-wikipedia-dataset

In [ ]:
import json

# データのロード
dataset = json.load(open('ner-wikipedia-dataset/ner.json','r'))

データの形式は以下のようになっています

In [ ]:
dataset[0:5]

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb
import unicodedata

# 固有表現のタイプとIDを対応付る辞書 
type_id_dict = {
    "人名": 1,
    "法人名": 2,
    "政治的組織名": 3,
    "その他の組織名": 4,
    "地名": 5,
    "施設名": 6,
    "製品名": 7,
    "イベント名": 8
}

# カテゴリーをラベルに変更、文字列の正規化する。
for sample in dataset:
    sample['text'] = unicodedata.normalize('NFKC', sample['text'])
    for e in sample["entities"]:
        e['type_id'] = type_id_dict[e['type']]
        del e['type']

In [ ]:
dataset[0:5]

In [ ]:
import random

# データセットの分割
random.seed(42)
random.shuffle(dataset)
n = len(dataset)
n_train = int(n*0.6)
n_val = int(n*0.2)
dataset_train = dataset[:n_train]
dataset_val = dataset[n_train:n_train+n_val]
dataset_test = dataset[n_train+n_val:]

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb
def create_dataset(tokenizer, dataset, max_length):
    """
    データセットをデータローダに入力できる形に整形。
    """
    input_ids = []
    token_type_ids = []
    attention_mask = []
    labels= []
    
    for sample in dataset:
        text = sample['text']
        entities = sample['entities']
        encoding = tokenizer.encode_plus_tagged(
            text, entities, max_length=max_length
        )
        input_ids.append(encoding['input_ids'])
        token_type_ids.append(encoding['token_type_ids'])
        attention_mask.append(encoding['attention_mask'])
        labels.append(encoding['labels'])
    
    d = {
        "input_ids": input_ids,
        "token_type_ids": token_type_ids,
        "attention_mask": attention_mask, 
        "labels": labels
    }
        
    return d

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb
from transformers import BertJapaneseTokenizer


class NER_tokenizer(BertJapaneseTokenizer):
       
    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。
        """
        # 固有表現の前後でtextを分割し、それぞれのラベルをつけておく。
        entities = sorted(entities, key=lambda x: x['span'][0])
        splitted = [] # 分割後の文字列を追加していく
        position = 0
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]
            label = entity['type_id']
            # 固有表現ではないものには0のラベルを付与
            splitted.append({'text':text[position:start], 'label':0}) 
            # 固有表現には、固有表現のタイプに対応するIDをラベルとして付与
            splitted.append({'text':text[start:end], 'label':label}) 
            position = end
        splitted.append({'text': text[position:], 'label':0})
        splitted = [ s for s in splitted if s['text'] ] # 長さ0の文字列は除く

        # 分割されたそれぞれの文字列をトークン化し、ラベルをつける。
        tokens = [] # トークンを追加していく
        labels = [] # トークンのラベルを追加していく
        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']
            tokens_splitted = tokenizer.tokenize(text)
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        encoding = tokenizer.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length', 
            truncation=True
        ) # input_idsをencodingに変換
        # 特殊トークン[CLS]、[SEP]のラベルを0にする。
        labels = [0] + labels[:max_length-2] + [0] 
        # 特殊トークン[PAD]のラベルを0にする。
        labels = labels + [0]*( max_length - len(labels) ) 
        encoding['labels'] = labels

        return encoding

    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。
        """
        # 文章のトークン化を行い、
        # それぞれのトークンと文章中の文字列を対応づける。
        tokens = [] # トークンを追加していく。
        tokens_original = [] # トークンに対応する文章中の文字列を追加していく。
        words = self.word_tokenizer.tokenize(text) # MeCabで単語に分割
        for word in words:
            # 単語をサブワードに分割
            tokens_word = self.subword_tokenizer.tokenize(word) 
            tokens.extend(tokens_word)
            if tokens_word[0] == '[UNK]': # 未知語への対応
                tokens_original.append(word)
            else:
                tokens_original.extend([
                    token.replace('##','') for token in tokens_word
                ])

        # 各トークンの文章中での位置を調べる。（空白の位置を考慮する）
        position = 0
        spans = [] # トークンの位置を追加していく。
        for token in tokens_original:
            l = len(token)
            while 1:
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += l
                    break

        # 符号化を行いBERTに入力できる形式にする。
        input_ids = tokenizer.convert_tokens_to_ids(tokens) 
        encoding = tokenizer.prepare_for_model(
            input_ids, 
            max_length=max_length, 
            padding='max_length' if max_length else False, 
            truncation=True if max_length else False
        )
        sequence_length = len(encoding['input_ids'])
        # 特殊トークン[CLS]に対するダミーのspanを追加。
        spans = [[-1, -1]] + spans[:sequence_length-2] 
        # 特殊トークン[SEP]、[PAD]に対するダミーのspanを追加。
        spans = spans + [[-1, -1]] * ( sequence_length - len(spans) ) 

        # 必要に応じてtorch.Tensorにする。
        if return_tensors == 'pt':
            encoding = { k: torch.tensor([v]) for k, v in encoding.items() }

        return encoding, spans

    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。
        """
        # labels, spansから特殊トークンに対応する部分を取り除く
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 同じラベルが連続するトークンをまとめて、固有表現を抽出する。
        entities = []
        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):
            
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities

In [ ]:
tokenizer_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# トークナイザのロード
tokenizer = NER_tokenizer.from_pretrained(tokenizer_name)

In [ ]:
# データセットの作成
max_length = 128

dataset_train = create_dataset(
    tokenizer, 
    dataset_train, 
    max_length
)

dataset_val = create_dataset(
    tokenizer, 
    dataset_val, 
    max_length
)

In [ ]:
import datasets

dataset_train = datasets.Dataset.from_dict(dataset_train)
dataset_val = datasets.Dataset.from_dict(dataset_val)

In [ ]:
dataset_train

In [ ]:
dataset_val

In [ ]:
# set format for pytorch
dataset_train.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])
dataset_val.set_format('torch', columns=['input_ids', 'attention_mask', 'token_type_ids', 'labels'])

In [ ]:
dataset_train[0]

## Uploading data to `sagemaker_session_bucket`

S3へデータをアップロードします。

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem


s3_prefix = 'samples/datasets/ner-wikipedia-dataset-io'
s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
dataset_train.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
dataset_val.save_to_disk(test_input_path, fs=s3)

In [ ]:
# 以下のpathにdatasetがuploadされました
print(training_input_path)
print(test_input_path)

# Fine-tuning & starting Sagemaker Training Job

`HuggingFace`のトレーニングジョブを作成するためには`HuggingFace` Estimatorが必要になります。    
Estimatorは、エンドツーエンドのAmazonSageMakerトレーニングおよびデプロイタスクを処理します。 Estimatorで、どのFine-tuningスクリプトを`entry_point`として使用するか、どの`instance_type`を使用するか、どの`hyperparameters`を渡すかなどを定義します。


```python
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    base_job_name='huggingface-sdk-extension',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    transformers_version='4.4',
    pytorch_version='1.6',
    py_version='py36',
    role=role,
    hyperparameters={
        'epochs': 1,
        'train_batch_size': 32,
        'model_name':'distilbert-base-uncased'
    }
)
```

SageMakerトレーニングジョブを作成すると、SageMakerは`huggingface`コンテナを実行するために必要なec2インスタンスの起動と管理を行います。    
Fine-tuningスクリプト`train.py`をアップロードし、`sagemaker_session_bucket`からコンテナ内の`/opt/ml/input/data`にデータをダウンロードして、トレーニングジョブを実行します。


```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

`HuggingFace estimator`で定義した`hyperparameters`は、名前付き引数として渡されます。

またSagemakerは、次のようなさまざまな環境変数を通じて、トレーニング環境に関する有用なプロパティを提供しています。

* `SM_MODEL_DIR`：トレーニングジョブがモデルアーティファクトを書き込むパスを表す文字列。トレーニング後、このディレクトリのアーティファクトはモデルホスティングのためにS3にアップロードされます。

* `SM_NUM_GPUS`：ホストで使用可能なGPUの数を表す整数。

* `SM_CHANNEL_XXXX`：指定されたチャネルの入力データを含むディレクトリへのパスを表す文字列。たとえば、HuggingFace estimatorのfit呼び出しで`train`と`test`という名前の2つの入力チャネルを指定すると、環境変数`SM_CHANNEL_TRAIN`と`SM_CHANNEL_TEST`が設定されます。

このトレーニングジョブをローカル環境で実行するには、`instance_type='local'`、GPUの場合は`instance_type='local_gpu'`で定義できます。    
**_Note：これはSageMaker Studio内では機能しません_**

In [ ]:
# トレーニングジョブで実行されるコード
!pygmentize ./scripts/train.py

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={
    'epochs': 5,
    'train_batch_size': 32,
    'eval_batch_size': 256,
    'learning_rate' : 1e-5,
    'model_name':'cl-tohoku/bert-base-japanese-whole-word-masking',
    'output_dir':'/opt/ml/checkpoints',
    'num_labels': 9,
}

# s3 uri where our checkpoints will be uploaded during training
job_name = "bert-ner-io"
#checkpoint_s3_uri = f's3://{sess.default_bucket()}/{job_name}/checkpoints'

# Creating an Estimator and start a training job

In [ ]:
huggingface_estimator = HuggingFace(
    entry_point='train.py',
    source_dir='./scripts',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    base_job_name=job_name,
    #checkpoint_s3_uri=checkpoint_s3_uri,
    #use_spot_instances=True,
    #max_wait=7200, # This should be equal to or greater than max_run in seconds'
    #max_run=3600, # expected max run in seconds
    role=role,
    transformers_version='4.6',
    pytorch_version='1.7',
    py_version='py36',
    hyperparameters=hyperparameters,
)

In [ ]:
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

# ml.p3.2xlarge, 5 epochでの実行時間の目安
#Training seconds: 558
#Billable seconds: 558

# Estimator Parameters

In [ ]:
# container image used for training job
print(f"container image used for training job: \n{huggingface_estimator.image_uri}\n")

# s3 uri where the trained model is located
print(f"s3 uri where the trained model is located: \n{huggingface_estimator.model_data}\n")

# latest training job name for this estimator
print(f"latest training job name for this estimator: \n{huggingface_estimator.latest_training_job.name}\n")

In [ ]:
# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

# Download-fine-tuned-model-from-s3

In [ ]:
import os

OUTPUT_DIR = './output/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=huggingface_estimator.model_data, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [ ]:
# OUTPUT_DIRに解凍します

!tar -zxvf model.tar.gz -C output

## Named Entity Recognition on Local

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb
import torch

def predict(text, tokenizer, bert_tc):
    """
    BERTで固有表現抽出を行うための関数。
    """
    # 符号化
    encoding, spans = tokenizer.encode_plus_untagged(
        text, return_tensors='pt'
    )
    #encoding = { k: v.cuda() for k, v in encoding.items() } # GPUで推論する場合

    # ラベルの予測値の計算
    with torch.no_grad():
        output = bert_tc(**encoding)
        scores = output.logits
        labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist() 

    # ラベル列を固有表現に変換
    entities = tokenizer.convert_bert_output_to_entities(
        text, labels_predicted, spans
    )

    return entities

In [ ]:
from transformers import AutoModelForTokenClassification

tokenizer_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = NER_tokenizer.from_pretrained(tokenizer_name)
model = AutoModelForTokenClassification.from_pretrained('./output')
# model = model.cuda() # GPUで推論する場合

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb

import itertools
from tqdm import tqdm

# 固有表現抽出
# 注：以下ではコードのわかりやすさのために、1データづつ処理しているが、
# バッチ化して処理を行った方が処理時間は短い
original_text=[]
entities_list = [] # 正解の固有表現を追加していく。
entities_predicted_list = [] # 抽出された固有表現を追加していく。

for sample in tqdm(dataset_test):
    text = sample['text']
    original_text.append(text)
    entities_predicted = predict(text, tokenizer, model) # BERTで予測
    entities_list.append(sample['entities'])
    entities_predicted_list.append( entities_predicted )

In [ ]:
print("テキスト: ", original_text[0])
print("正解: ", entities_list[0])
print("抽出: ", entities_predicted_list[0])

# Evaluate NER model

In [ ]:
# https://github.com/stockmarkteam/bert-book/blob/master/Chapter8.ipynb

def evaluate_model(entities_list, entities_predicted_list, type_id=None):
    """
    正解と予測を比較し、モデルの固有表現抽出の性能を評価する。
    type_idがNoneのときは、全ての固有表現のタイプに対して評価する。
    type_idが整数を指定すると、その固有表現のタイプのIDに対して評価を行う。
    """
    num_entities = 0 # 固有表現(正解)の個数
    num_predictions = 0 # BERTにより予測された固有表現の個数
    num_correct = 0 # BERTにより予測のうち正解であった固有表現の数

    # それぞれの文章で予測と正解を比較。
    # 予測は文章中の位置とタイプIDが一致すれば正解とみなす。
    for entities, entities_predicted in zip(entities_list, entities_predicted_list):

        if type_id:
            entities = [ e for e in entities if e['type_id'] == type_id ]
            entities_predicted = [ 
                e for e in entities_predicted if e['type_id'] == type_id
            ]
            
        get_span_type = lambda e: (e['span'][0], e['span'][1], e['type_id'])
        set_entities = set(get_span_type(e) for e in entities)
        set_entities_predicted = set(get_span_type(e) for e in entities_predicted)

        num_entities += len(entities)
        num_predictions += len(entities_predicted)
        num_correct += len( set_entities & set_entities_predicted )

    # 指標を計算
    precision = num_correct/num_predictions # 適合率
    recall = num_correct/num_entities # 再現率
    f_value = 2*precision*recall/(precision+recall) # F値

    result = {
        'num_entities': num_entities,
        'num_predictions': num_predictions,
        'num_correct': num_correct,
        'precision': precision,
        'recall': recall,
        'f_value': f_value
    }

    return result

In [ ]:
print(evaluate_model(entities_list, entities_predicted_list))